## Spam Classification Model (Sklearn)

- Wrap a ML model for use as a prediction microservice in seldon-core
- Run locally on Docker to test
- Deploy on seldon-core running on k8s cluster

### Train Locally

In [20]:
import numpy as np 
import pandas as pd
from sklearn.externals import joblib
from pathlib import Path
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
model_path: Path=Path('./')

In [14]:
data = pd.read_csv("spam.csv",encoding='latin-1')
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"class", "v2":"text"})
data.head()

def pre_process(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

features = data['text'].copy()
features = features.apply(pre_process)

vectorizer = TfidfVectorizer("english")
_features = vectorizer.fit_transform(features)
with open('Spam-Classifier/model/vectorizer.pkl', 'wb') as vect:
    pickle.dump(vectorizer, vect)

In [15]:
vectorizer = joblib.load(model_path.joinpath('Spam-Classifier/model/vectorizer.pkl'))
train_x, test_x, train_y, test_y = train_test_split(_features, data['class'], test_size=0.3, random_state=0)
svc = SVC(kernel='sigmoid', gamma=1.0, probability=True)
svc.fit(train_x,train_y)
# save the model to disk
filename = 'Spam-Classifier/model/model.pkl'
pickle.dump(svc, open(filename, 'wb'))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1.0, kernel='sigmoid',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [16]:
clf = joblib.load(model_path.joinpath(filename))

In [21]:
prediction = clf.predict(test_x)
accuracy_score(test_y,prediction)

0.9730861244019139

In [22]:
message = np.array(['click here to win the price'])
data = vectorizer.transform(message).todense()
probas = clf.predict_proba(data)
probas

array([[0.0220629, 0.9779371]])

In [53]:
clf.classes_

array(['ham', 'spam'], dtype=object)

### wrap each model component using s2i

In [43]:
!s2i build Spam-Classifier/ seldonio/seldon-core-s2i-python3:1.1.1-rc spam-classifier:1.0.0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [49]:
!docker run --name "spam-classifier" -d --rm -p 5000:5000 spam-classifier:1.0.0.1

1b8159f67b7ddbd2de26833411303ebee8e08331097e28754f04688c1fb86d3c


In [51]:
!curl -g http://localhost:5000/predict --data-urlencode 'json={"data": {"names": ["message"], "ndarray": ["click here to win the price"]}}'


{"data":{"ndarray":["0.9779371008528993","spam"]},"meta":{}}


In [52]:
!docker rm spam-classifier --force

spam-classifier


In [30]:
!s2i build Translator/ seldonio/seldon-core-s2i-python3:1.1.1-rc translator:1.0.0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Running setup.py bdist_wheel for goslate: started
Running setup.py bdist_wheel for goslate: finished with status 'done'
Stored in directory: /root/.cache/pip/wheels/4f/7f/28/6f52271012a7649b54b1a7adaae329b4246bbbf9d1e4f6e51a
Successfully built goslate
Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [55]:
!docker run --name "eng-translator" -d --rm -p 5000:5000 translator:1.0.0.1

ca18617eed4ee5b12c1ce835d94a677007e3c095166b8e4e5d0f9fd164757814


In [57]:
!curl -g http://localhost:5000/transform-input --data-urlencode 'json={"data": {"names": ["message"], "ndarray": ["Wie läuft dein Tag"]}}'

{"data":{"names":["message"],"ndarray":["How is your day"]},"meta":{}}


In [58]:
!docker rm eng-translator --force

eng-translator


Assuming you have kubernetes cluster running and seldon-core installed, you can deploy your Machine Learning model using:

kubectl apply -f deploy.yaml